In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
traindata_gen = ImageDataGenerator(rescale = 1/255,
                                  zoom_range=0.2,
                                  shear_range=0.3,
                                  rotation_range=90,
                                  horizontal_flip=True)
testdata_gen = ImageDataGenerator(rescale = 1/255)

In [ ]:
train_data = traindata_gen.flow_from_directory('../input/catdogdifferenciator/dataset/training_set',
                                              target_size = (128, 128),
                                              color_mode='rgb', batch_size=32)
# val_data = testdata_gen.flow_from_directory(r'C:\Users\PIYUSH KUMAR\Desktop\ineuron\practice\catdog\dataset\val_set',
#                                            target_size = (150, 150),
#                                            color_mode = 'rgb', batch_size=8)
test_data = testdata_gen.flow_from_directory('../input/catdogdifferenciator/dataset/test_set',
                                            target_size = (128, 128),
                                            color_mode = 'rgb', batch_size=32)

# Xception keras model

In [ ]:
xcpn = keras.applications.Xception(include_top = False, weights = 'imagenet', input_shape = (128, 128, 3))
for layer in xcpn.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(xcpn.output)
x = layers.Dense(1024, activation = 'relu')(x)
prediction = layers.Dense(2, activation = 'softmax')(x)

In [ ]:
model_xcpn = keras.models.Model(inputs = xcpn.input, outputs = prediction)
# model_xcpn.summary()

In [ ]:
model_xcpn.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, mode = 'min')
cp = keras.callbacks.ModelCheckpoint('best_model.h5', monitor = 'val_accuracy', mode = 'max', save_best_only = True)

In [ ]:
history = model_xcpn.fit(train_data, epochs = 50, validation_data = test_data, callbacks=[es, cp])

# Prediction

In [ ]:
path = '../input/catdogdifferenciator/dataset/test_set/cats/cat.4060.jpg'
img = image.load_img(path, target_size = (128, 128))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis = 0)
img_preprocess = preprocess_input(img_batch)
prediction = model_xcpn.predict(img_preprocess)
plt.imshow(img)
plt.title(prediction)

In [ ]:
print(model_xcpn.evaluate(test_data))